#Nhận dạng thực thể sử dụng thư viện Flair để xây dựng mô hình học sâu BiLSTM_CRF#
Ở bài trước ta đã thiết kế các đặc trưng và sử dụng mô hình CRF để huấn luyện. Ở bài này ta sẽ tiến hành sử xây dựng một mô hình học sâu theo kiến trúc Glove+BiLSTM+CRF để thực hiện bài toán NER

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Cài thư viện
!pip install flair==0.11.1
!pip install sacremoses

In [3]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import TransformerWordEmbeddings,TransformerDocumentEmbeddings
from flair.datasets import CONLL_03, NER_ENGLISH_MOVIE_SIMPLE
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

columns = {0: 'text',1:'pos',2:'pharse', 3: 'ner'}


# this is the folder in which train, test and dev files reside
data_folder = '/content/drive/MyDrive/NLP/Đáp án cho bài tập thực hành/Lesson6/data'

# init a corpus using column format, data folder and the names of the train, dev and test files
##START CODE HERE##
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.conll',
                              test_file='test.conll')
##END CODE HERE##
label_type = 'ner'

# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)

# 4. initialize embedding stack with GloVe
##START CODE HERE##
embedding_types = [
    WordEmbeddings('glove')
]
##END CODE HERE##


embeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger 
##START CODE HERE##
tagger = SequenceTagger(embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True,
                        bidirectional=True,    
                        rnn_type='GRU',
                        )
##END CODE HERE##
# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('content/model',
              learning_rate=0.1,
              mini_batch_size=4,
              max_epochs=3,
              )

2022-10-07 02:20:33,179 Reading data from /content/drive/MyDrive/NLP/Đáp án cho bài tập thực hành/Lesson6/data
2022-10-07 02:20:33,180 Train: /content/drive/MyDrive/NLP/Đáp án cho bài tập thực hành/Lesson6/data/train.conll
2022-10-07 02:20:33,184 Dev: None
2022-10-07 02:20:33,185 Test: /content/drive/MyDrive/NLP/Đáp án cho bài tập thực hành/Lesson6/data/test.conll
2022-10-07 02:20:44,303 Computing label dictionary. Progress:


6989it [00:00, 36041.50it/s]

2022-10-07 02:20:44,573 Dictionary created for label 'ner' with 5 values: PERSON (seen 2443 times), LOCATION (seen 2128 times), ORGANIZATION (seen 1472 times), DATETIME (seen 635 times)


2022-10-07 02:20:45,197 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpggkpet41


100%|██████████| 160000128/160000128 [00:07<00:00, 20961362.00B/s]

...

2022-10-07 02:29:58,558 ----------------------------------------------------------------------------------------------------
2022-10-07 02:29:58,569 loading file content/model/best-model.pt
2022-10-07 02:29:59,324 SequenceTagger predicts: Dictionary with 19 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-LOCATION, B-LOCATION, E-LOCATION, I-LOCATION, S-ORGANIZATION, B-ORGANIZATION, E-ORGANIZATION, I-ORGANIZATION, S-DATETIME, B-DATETIME, E-DATETIME, I-DATETIME, <START>, <STOP>


100%|██████████| 309/309 [00:08<00:00, 35.34it/s]

2022-10-07 02:30:08,152 Evaluating as a multi-label problem: False


2022-10-07 02:30:08,183 0.6531	0.1419	0.2332	0.1329
2022-10-07 02:30:08,192 
Results:
- F-score (micro) 0.2332
- F-score (macro) 0.213
- Accuracy 0.1329

By class:
              precision    recall  f1-score   support

    LOCATION     0.7471    0.1477    0.2466       860
      PERSON     0.7389    0.1861    0.2973       806
ORGANIZATION     0.4217    0.1124    0.1774       623
    DATETIME     0.8462    0.0707    0.1306       311

   micro avg     0.6531    0.1419    0.2332      2600
   macro avg     0.6885    0.1292    0.2130      2600
weighted avg     0.6784    0.1419    0.2319      2600

2022-10-07 02:30:08,195 ----------------------------------------------------------------------------------------------------


{'test_score': 0.23317535545023696,
 'dev_score_history': [0.14722004698512137,
  0.23541453428863868,
  0.2901960784313726],
 'train_loss_history': [0.3096780628129557,
  0.25853154231230335,
  0.2426939838638497],
 'dev_loss_history': [0.4363804757595062,
  0.22578659653663635,
  0.21106714010238647]}

#Predict Model

In [5]:
from flair.models import SequenceTagger
from flair.data import Sentence
import json

#Load Model


path_model = '/content/drive/MyDrive/NLP/Đáp án cho bài tập thực hành/Lesson6/data/model/best-model.pt'
path_raw_text = '/content/drive/MyDrive/NLP/Đáp án cho bài tập thực hành/Lesson6/data/data-thuc-the.txt'
path_write_file = '/content/drive/MyDrive/NLP/Đáp án cho bài tập thực hành/Lesson6/data/predict.json'

# Load model
tagger = SequenceTagger.load(path_model)


f_raw_text = open(path_raw_text,'r',encoding='utf-8')
f_write = open(path_write_file,'w',encoding='utf-8')
lines = f_raw_text.readlines()

# Tiến hành đọc các raw_text từ path_raw_text, sử dụng mô hình để in ra nhãn đoán, lưu file kết quả dưới dạng json 
## START CODE HERE
lst_json = []
for line in lines:
    obj = {}
    if line=='\n':
        continue
    sentence = Sentence(line)
    tagger.predict(sentence)
    obj['text'] = line
    obj['entities'] = []
    for entity in sentence.get_spans('ner'):
        obj_ner = {}
        text = entity.text
        ner_tag = entity.get_label("ner").value
        obj_ner['text'] = entity.text
        obj_ner['tag'] = ner_tag
        obj_ner['start_position'] = entity.start_position
        obj_ner['end_position'] = entity.end_position
        obj_ner['score'] = entity.get_label("ner").score
        obj['entities'].append(obj_ner)
    lst_json.append(obj)
json.dump(lst_json, f_write,indent=4,ensure_ascii = False)
f_raw_text.close()
f_write.close()
## END CODE HERE

2022-10-07 02:32:22,355 loading file /content/content/model/best-model.pt
2022-10-07 02:32:22,912 SequenceTagger predicts: Dictionary with 19 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-LOCATION, B-LOCATION, E-LOCATION, I-LOCATION, S-ORGANIZATION, B-ORGANIZATION, E-ORGANIZATION, I-ORGANIZATION, S-DATETIME, B-DATETIME, E-DATETIME, I-DATETIME, <START>, <STOP>
